<a href="https://colab.research.google.com/github/kyledinh/gsb-mlops/blob/main/labs/codellama-13a/nb_fine_codellama_13b_gsb_repea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Finetuning Using Slowblood Package (DEV)

- Install Packages
- Settings and Vars
- Load Quantized Models
- Tokenizer
- Peft
- Load Dataset

In [16]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
%pip freeze

anyio==4.1.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==23.1.0
Babel==2.13.1
beautifulsoup4==4.12.2
bleach==6.1.0
blinker==1.4
certifi==2023.11.17
cffi==1.16.0
charset-normalizer==3.3.2
comm==0.2.0
cryptography==3.4.8
dbus-python==1.2.18
debugpy==1.8.0
decorator==5.1.1
defusedxml==0.7.1
distro==1.7.0
entrypoints==0.4
exceptiongroup==1.2.0
executing==2.0.1
fastjsonschema==2.19.0
filelock==3.13.1
fqdn==1.5.1
fsspec==2023.10.0
httplib2==0.20.2
idna==3.6
importlib-metadata==4.6.4
ipykernel==6.26.0
ipython==8.18.1
ipython-genutils==0.2.0
ipywidgets==8.1.1
isoduration==20.11.0
jedi==0.19.1
jeepney==0.7.1
Jinja2==3.1.2
json5==0.9.14
jsonpointer==2.4
jsonschema==4.20.0
jsonschema-specifications==2023.11.1
jupyter-archive==3.4.0
jupyter-contrib-core==0.4.2
jupyter-contrib-nbextensions==0.7.0
jupyter-events==0.9.0
jupyter-highlight-selected-word==0.2.0
jupyter-lsp==2.2.1
jupyter-nbextensions-configurator==0.6.3
jupyter_client==7.4.9
jup

In [1]:
%pip install --upgrade pip
%pip install -qU git+https://github.com/huggingface/accelerate.git
%pip install -qU git+https://github.com/huggingface/peft.git
%pip install -qU git+https://github.com/huggingface/transformers.git
# !pip install transformers==4.31 #temporary fix required owing to breaking changes on Aug 9th 2023
%pip install -qU bitsandbytes
%pip install -qU datasets
%pip install -qU einops
%pip install -qU flash-attn --no-build-isolation
%pip install -qU huggingface_hub
%pip install -qU langchain
%pip install -qU pytesseract
%pip install -qU pypdfium2
%pip install -qU safetensors
%pip install -qU torch
%pip install -qU xformers

# %pip install -q -U slowblood
## IMPORTING THE UPLOADED module from pypi/src/slowblood to /workspace/slowblood
import os
os.chdir(os.path.expanduser("/workspace"))
import slowblood
print("Loaded local VERSION: ", slowblood.lib_settings.VERSION)

import huggingface_hub
import torch
import transformers

from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# LOGIN TO HUGGINGFACE WITH ACCES TOKEN
huggingface_hub.notebook_login()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Loaded loc

# Settings and Vars

- SETTINGS, configuration for notebook

In [2]:
import importlib
importlib.reload(slowblood)
print("version: ", slowblood.lib_settings.VERSION)

%pip freeze | grep slowblood

version:  2
Note: you may need to restart the kernel to use updated packages.


In [ ]:
SETTINGS = slowblood.FineTuningSettings(
  base_model="codellama/CodeLlama-13b-Instruct-hf",
  new_model_name="gsb-repea-codellama-13b-ft",
  repo_name="Slowblood",
  dataset="Slowblood/gsb-rapid-entry-pea-v2",
  cache_dir="/workspace/cache"
)

print(SETTINGS.new_model)
print(SETTINGS.adapter_model)
print("VERSION: ", slowblood.lib_settings.VERSION)

In [ ]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# Load Quantized Model

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Now you can create the model with the modified configuration
model = AutoModelForCausalLM.from_pretrained(
    SETTINGS.base_model,
    quantization_config=bnb_config,
    device_map={"":0},
    # device_map="cpu",
    trust_remote_code=True,
    cache_dir=SETTINGS.cache_dir
)

In [ ]:
print(transformers.__version__)
!pip show bitsandbytes

## Set Up Tokenizer

Get tokenizer, then set pad token, update the model

In [ ]:
tokenizer = slowblood.getTokenizerForModel(SETTINGS.base_model, cache_dir=SETTINGS.cache_dir, debug=True)
model = slowblood.updateModelWithTokenizer(model, tokenizer, debug=True)

## Set up LoRa

In [ ]:
# #only if you want to load with adapters (it can be faster to download a base model and add adapters)
# from peft import PeftModel

# # adapter_model = f'{base_model}' + '-function-calling-adapters-v2'
# adapter_model = "RonanMcGovern/Llama-2-13b-chat-hf-function-calling-adapters-v2"

# # load peft model with adapters
# model = PeftModel.from_pretrained(
#     model,
#     adapter_model,
# )

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print(model.state_dict().keys())

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
      "self_attn.q_proj",
      "self_attn.k_proj",
      "self_attn.v_proj",
      "self_attn.o_proj",
      "self_attn.rotary_emb.inv_freq",
      "mlp.gate_proj",
      "mlp.up_proj",
      "mlp.down_proj",
      "input_layernorm.weight",
      "post_attention_layernorm.weight",
      "model.norm.weight",
      "lm_head.weight"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Prepare Data

In [ ]:
%pip install -q -U datasets

In [ ]:
from datasets import load_dataset

data = load_dataset(
    SETTINGS.dataset # "Slowblood/gsb-rapid-entry-pea",
    # revision="functionList" # optionally specify a branch
    )

In [ ]:
print(data)

In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings, response_lengths, input_lengths):
        self.encodings = encodings
        self.response_lengths = response_lengths
        self.input_lengths = input_lengths

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}

        # Set labels to be the same as input_ids
        item["labels"] = item["input_ids"].clone()

        # Calculate the start and end positions of the response
        response_start_position = self.input_lengths[idx]
        response_end_position = self.input_lengths[idx] + self.response_lengths[idx]

        # Create a loss mask that covers only the response tokens
        item["loss_mask"] = torch.zeros_like(item["input_ids"])
        item["loss_mask"][response_start_position:response_end_position] = 1

        # Shift the loss mask to the left by one position
        shifted_loss_mask = torch.cat([item["loss_mask"][1:], torch.tensor([0])])
        item["loss_mask"] = shifted_loss_mask

        # Shift the labels to the left by one position
        item["labels"][:-1] = item["input_ids"][1:]

        # Replace the token after the response with an EOS token
        item["labels"][response_end_position - 1] = 2

        # Replace the token after the response with an 1 in the loss mask
        item["loss_mask"][response_end_position - 1] = 1

        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
def prepare_dataset(dataset, tokenizer):
    # Define the roles and markers
    B_FUNC, E_FUNC = "<FUNCTIONS>", "</FUNCTIONS>"
    B_SYS, E_SYS = "<SYS>", "</SYS>"
    B_USER, E_USER = "<USER>", "</USER>"
    B_INST, E_INST = "[INST]", "[/INST]"

    # Create the formatted text with the correct roles for each part of the dialogue
    formatted_dataset = dataset.map(
        lambda x: {
            "input_text": "".join([
                f"{B_SYS}{x['systemPrompt'].strip()}{E_SYS}",
                f"{B_INST} {x['userPrompt'].strip()} {E_INST}\n\n",
                f"{x['assistantResponse'].strip()}",  # appending the EOS token in TextData...
            ]),
            "response_text": "".join([
                f"{x['assistantResponse'].strip()}",  # appending the EOS token in TextData...
            ]),
        }
    )

    # Tokenize the datasets
    encodings = tokenizer([dialogue["input_text"] for dialogue in formatted_dataset], truncation=True, padding=True, max_length=1024, return_tensors='pt', add_special_tokens=True)

    # Tokenize the response one by one without padding and special tokens for the purpose of calculating length
    response_lengths = [len(tokenizer.encode(dialogue["response_text"], truncation=True, max_length=1024, padding=False, add_special_tokens=False)) for dialogue in formatted_dataset]

    # Tokenize the input one by one without padding and with the initial special token for the purpose of calculating length
    total_lengths = [len(tokenizer.encode(dialogue["input_text"], truncation=True, max_length=1024, padding=False, add_special_tokens=True)) for dialogue in formatted_dataset]
    input_lengths = [total_length - response_length for total_length, response_length in zip(total_lengths, response_lengths)]

    # Create TextDataset
    text_dataset = TextDataset(encodings, response_lengths, input_lengths)

    return text_dataset

In [ ]:
# Apply function to your datasets
train_dataset = prepare_dataset(data['train'], tokenizer)
test_dataset = prepare_dataset(data['test'], tokenizer)

### Examine the datasets

In [ ]:
# Print the number of items in the dataset
print(f"Number of samples in the dataset: {len(train_dataset)}")

# Get a sample item
sample_item = train_dataset[11]  # replace 0 with the index of the sample you want to examine

# Print the dimensions of the sample item
print(f"Dimensions of input_ids: {sample_item['input_ids'].shape}")
print(f"Dimensions of attention_mask: {sample_item['attention_mask'].shape}")
print(f"Dimensions of loss_mask: {sample_item['loss_mask'].shape}")
print(f"Dimensions of labels: {sample_item['labels'].shape}")

# Print some tokens from the start and end of the sample
num_tokens_to_print = 336  # replace with the number of tokens you want to print

print("\nTokens at the start of the sample:")
print(sample_item['input_ids'][:num_tokens_to_print].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['input_ids'][:num_tokens_to_print].tolist()))

print("\nLabels at the start of the sample:")
print(sample_item['labels'][:num_tokens_to_print].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['labels'][:num_tokens_to_print].tolist()))

print("Attention mask at the start of the sample:")
print(sample_item['attention_mask'][:num_tokens_to_print].tolist())

print("Loss mask at the start of the sample:")
print(sample_item['loss_mask'][:num_tokens_to_print].tolist())

print("\nTokens at the end of the sample:")
print(sample_item['input_ids'][-num_tokens_to_print:].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['input_ids'][-num_tokens_to_print:].tolist()))

print("\nLabels at the end of the sample:")
print(sample_item['labels'][-num_tokens_to_print:].tolist())
print(tokenizer.convert_ids_to_tokens(sample_item['labels'][-num_tokens_to_print:].tolist()))

print("Attention mask at the end of the sample:")
print(sample_item['attention_mask'][-num_tokens_to_print:].tolist())

print("Loss mask at the end of the sample:")
print(sample_item['loss_mask'][-num_tokens_to_print:].tolist())


## Generate a sample

In [ ]:
import textwrap
wrapper = textwrap.TextWrapper(width=80)

In [ ]:
import re  # import regular expressions module

In [ ]:
def generate(index):
    # Define the roles and markers
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<SYS>", "</SYS>\n\n"
    B_FUNC, E_FUNC = "<FUNCTIONS>", "</FUNCTIONS>\n\n"

    # functionList = data['test'][index]['functionList']
    # user_prompt = data['test'][index]['userPrompt']
    # correct_answer = data['test'][index]['assistantResponse']

    # Format your prompt template
    system_prompt = data['train'][index]['systemPrompt']
    user_prompt = data['train'][index]['userPrompt']
    correct_answer = data['train'][index]['assistantResponse']

    # Format your prompt template
    prompt = f"{B_SYS}{system_prompt.strip()}{E_SYS}{B_INST} {user_prompt.strip()} {E_INST}\n\n"

    print("Prompt:")
    print(prompt)

    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    output = model.generate(input_ids=encoding.input_ids,
                            attention_mask=encoding.attention_mask,
                            max_new_tokens=2000,
                            do_sample=True,
                            temperature=0.01,
                            eos_token_id=tokenizer.eos_token_id,
                            top_k=0)

    print()

    # Subtract the length of input_ids from output to get only the model's response
    output_text = tokenizer.decode(output[0, len(encoding.input_ids[0]):], skip_special_tokens=False)
    output_text = re.sub('\n+', '\n', output_text)  # remove excessive newline characters

    print("Generated Assistant Response:")
    print(output_text)

    print()

    print("Correct Assistant Response:")
    print(correct_answer)

    print()

In [ ]:
generate(0)

# Training

In [ ]:
import torch.nn as nn

In [ ]:
class CustomTrainer(transformers.Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Define the number of tokens you want to display
        num_tokens = 25  # This displays info on the actual and predicted tokens at the end of each sequence.

        labels = inputs.pop("labels")
        loss_mask = inputs.pop("loss_mask")

        # Forward pass
        outputs = model(**inputs)

        logits = outputs.logits

        # Check for NaN in logits and labels
        if torch.isnan(logits).any():
            print("NaN detected in logits")
            print(logits)

        # Convert logits to probabilities using softmax function
        probs = nn.functional.softmax(logits, dim=-1)

        # Get the most probable tokens
        predicted_token_ids = torch.argmax(probs, dim=-1)

        # Compute the loss
        loss_fct = nn.CrossEntropyLoss(reduction='none')
        losses = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))

        # Reshaping the losses to have dimensions [batch_size, seq_length]
        losses = losses.view(-1, inputs['input_ids'].size(1))

        # Apply the loss mask
        masked_loss = losses * loss_mask

        # Check for NaN in losses and zero in loss_mask.sum()
        if torch.isnan(losses).any():
            print("NaN detected in losses")
            # print(losses)

        if loss_mask.sum() == 0:
            print("Sum of loss_mask is zero")
            return (torch.tensor(0).to(loss_mask.device), outputs) if return_outputs else torch.tensor(0).to(loss_mask.device)  # Early return

        # Aggregate the masked losses
        loss = masked_loss.sum() / (loss_mask.sum() + 1e-9)  # normalizing by the number of tokens considered + epsilon to prevent division by zero

        # Print formatted tokens
        batch_size, seq_length = inputs['input_ids'].size()

        # num_tokens = len(inputs['input_ids'][0])

        # # Useful for debugging training - recommend just training a small number of steps
        # print("-" * 120)
        # print(f"Token analysis for last {num_tokens} tokens:")
        # header_format = "{:<10}{:<20}{:<20}{:<20}{:<20}{:<30}{:<30}".format("Index", "Input Token", "Predicted Token", "True Token", "Loss Mask", "Raw Loss", "Masked Loss")

        # for batch_idx in range(batch_size):
        #     input_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][batch_idx])  # Using batch_idx
        #     predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids[batch_idx])  # Using batch_idx
        #     true_tokens = tokenizer.convert_ids_to_tokens(labels[batch_idx])  # Using batch_idx

        #     print(f"\nBatch {batch_idx + 1} of {batch_size}:")
        #     print(header_format)
        #     for i in range(-num_tokens, 0, 1):
        #         index = seq_length + i  # Correct index based on sequence length
        #         print("{:<10}{:<20}{:<20}{:<20}{:<20.1f}{:<30.6f}{:<30.6f}".format(index, input_tokens[index], predicted_tokens[index], true_tokens[index], loss_mask[batch_idx, i].item(), losses[batch_idx, i], masked_loss[batch_idx, i]))
        #     print("-" * 120)

        return (loss, outputs) if return_outputs else loss

    def get_train_dataloader(self):
      train_dataset = self.train_dataset
      data_collator = self.data_collator

      dataloader_params = {
          "batch_size": self.args.train_batch_size,
          "collate_fn": data_collator,
          "num_workers": self.args.dataloader_num_workers,
          "pin_memory": self.args.dataloader_pin_memory,
      }

      if not isinstance(train_dataset, torch.utils.data.IterableDataset):
          dataloader_params["sampler"] = self._get_train_sampler()
          dataloader_params["drop_last"] = self.args.dataloader_drop_last

      return DataLoader(train_dataset, **dataloader_params)

    def get_eval_dataloader(self, eval_dataset=None):
      eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
      if eval_dataset is None:
          raise ValueError("Trainer: evaluation requires an eval_dataset.")

      data_collator = self.data_collator

      # Parameters for the DataLoader
      dataloader_params = {
          "batch_size": self.args.eval_batch_size,
          "collate_fn": data_collator,
          "num_workers": self.args.dataloader_num_workers,
          "pin_memory": self.args.dataloader_pin_memory,
      }

      # If your dataset isn't an instance of torch's IterableDataset, you can provide sampler and drop_last
      if not isinstance(eval_dataset, torch.utils.data.IterableDataset):
          dataloader_params["sampler"] = self._get_eval_sampler(eval_dataset)
          dataloader_params["drop_last"] = False  # Typically, you don't drop the last batch for evaluation

      return DataLoader(eval_dataset, **dataloader_params)

In [ ]:
class CustomDataCollator: # Needed if the EOS token is to be included in training.
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):

        input_ids = torch.stack([item['input_ids'] for item in batch])
        attention_mask = torch.stack([item['attention_mask'] for item in batch])
        labels = torch.stack([item['labels'] for item in batch])
        loss_mask = torch.stack([item['loss_mask'] for item in batch])

        # # Debugging: print details of the first sequence in the batch
        # num_elements_to_view = 20  # Number of last elements to view

        # # Decoding the input_ids
        # decoded_input_tokens = self.tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

        # print("Debugging last", num_elements_to_view, "elements of the first sequence in the batch:")
        # print("{:<20}{:<20}{:<20}{:<20}".format("Token", "Input ID", "Label", "Loss Mask"))
        # for i in range(-num_elements_to_view, 0, 1):
        #   print("{:<20}{:<20}{:<20}{:<20}".format(decoded_input_tokens[i], input_ids[0, i].item(), labels[0, i].item(), loss_mask[0, i].item()))

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'loss_mask': loss_mask
        }

data_collator = CustomDataCollator(tokenizer)


In [ ]:
trainer = CustomTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, #turn on the eval dataset if you want to use evaluation functionality versus the test dataset (not provided in this script)
    args=transformers.TrainingArguments(
        num_train_epochs=5,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        evaluation_strategy="steps",
        max_grad_norm=1,
        warmup_ratio=0.1,
        eval_steps=0.2,
        # max_steps=3,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=data_collator,
    # data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()
torch.cuda.empty_cache()

# Example After Fine Tuning

In [ ]:
model.config.use_cache = True
model.eval()

In [ ]:
generate(10)

In [ ]:
generate(1)

# Merge Adapters and Save Model to Hub

In [ ]:
def getTokenFromFile(file):
    with open(file, 'r') as file:
        data = file.read().rstrip()
    return data

HF_TOKEN = getTokenFromFile("/root/.cache/huggingface/token")

print(HF_TOKEN[0:9])
# Save the model
model.save_pretrained(SETTINGS.adapter_model, push_to_hub=True)

# Push the model to the hub
model.push_to_hub(SETTINGS.adapter_model, private=True, token=HF_TOKEN)

In [ ]:
# from transformers import AutoModelForCausalLM, PretrainedConfig
# import torch

# reload the base model (you might need a pro subscription for this because you may need a high RAM environment since this is loading the full original model, not quantized)
model = AutoModelForCausalLM.from_pretrained(SETTINGS.base_model, device_map='cpu', trust_remote_code=True, torch_dtype=torch.float16, cache_dir=SETTINGS.cache_dir)

In [ ]:
from peft import PeftModel

# load perf model with new adapters
model = PeftModel.from_pretrained(
    model,
    SETTINGS.adapter_model,
)

In [ ]:
model = model.merge_and_unload() # merge adapters with the base model.

In [ ]:
model.push_to_hub(SETTINGS.new_model, use_auth_token=True, max_shard_size="10GB")

In [ ]:
#Push the tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(SETTINGS.base_model, trust_remote_code=True)
tokenizer.push_to_hub(SETTINGS.new_model, use_auth_token=True)

# Known Issues/Improvements
- No open issues.